In [1]:
using Plots, ApproxFun, Interact
gr()  # Also works with plotly();

# $$u'' + 2u = \cos(\omega t)\qquad u(0)=u'(0)=0$$

Slide  to $\omega=\sqrt 2$ to see a resonance, where the solution grows

In [2]:
t=Fun(identity,0..1000) 
L=𝒟^2+2I  # our differential operator, 𝒟 is equivalent to Derivative()

@manipulate for ω=0.:.1sqrt(2):2sqrt(2)
    u=[ivp();L]\[0.;0.;cos(ω*t)]
    plot(u)
end

Interact.Slider{Float64}(Signal{Float64}(1.2727922061357857, nactions=0),"ω",1.2727922061357857,0.0:0.14142135623730953:2.687005768508881,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,363.937 596.063,364.034 596.062,364.226 596.06,364.514 596.058,364.894 596.056,365.365 596.053,365.922 596.05,366.564 596.046,367.283 596.042,368.077 
 596.037,368.938 596.032,369.861 596.026,370.837 596.02,371.859 596.013,372.919 596.006,374.006 595.999,375.11 595.991,376.221 595.982,377.326 595.973,378.413 
 595.964,379.468 595.954,380.478 595.943,381.426 595.932,382.298 595.921,383.077 595.909,383.746 595.897,384.286 595.884,384.678 595.871,384.904 595.857,384.943 
 595.843,384.775 595.828,384.379 595.813,383.733 595.797,382.816 595.781,381.606 595.765,380.081 595.748,378.219 595.73,375.998 595.712,373.397 595.694,370.396 
 595.675,366.975 595.656,363.114 595.636,358.796 595.615,354.004 595.595,348.723 595.573,342.941 595.551,336.648 595.529,329.834 595.507,322.494 595.483,314.626 
 595.46,306.232 595.436,297.315 595.411,287.884 595.386,277.952 595.36,267.536 595.334,256.658 595.308,245.346 595.281,233.632 595.253,221.552 595.226,209.151 
 595.197,196.478 595.168,183.586 595.139,170.536 595.109,157.394 595.079,144.232 595.048,131.126 595.017,118.158 594.985,105.414 594.953,92.9854 594.921,80.9659 
 594.888,69.4529 594.854,58.5462 594.82,48.3466 594.785,38.9557 594.75,30.4745 594.715,23.0023 594.679,16.6356 594.643,11.4665 594.606,7.58178 594.569,5.06133 
 594.531,3.97651 594.492,4.38886 594.454,6.34858 594.414,9.89306 594.375,15.0455 594.335,21.8134 594.294,30.1875 594.253,40.1407 594.211,51.6268 594.169,64.5799 
 594.127,78.914 594.084,94.5223 594.04,111.277 593.996,129.032 593.952,147.618 593.907,166.85 593.862,186.524 593.816,206.421 593.77,226.307 593.723,245.939 
 593.676,265.066 593.628,283.429 593.58,300.77 593.531,316.834 593.482,331.372 593.433,344.145 593.383,354.933 593.332,363.531 593.281,369.765 593.23,373.485 
 593.178,374.577 593.126,372.965 593.073,368.611 593.02,361.525 592.966,351.759 592.912,339.416 592.857,324.647 592.802,307.649 592.746,288.67 592.69,268 
 592.633,245.973 592.576,222.958 592.519,199.359 592.461,175.602 592.402,152.134 592.343,129.408 592.284,107.881 592.224,87.9972 592.164,70.1839 592.103,54.837 
 592.042,42.3128 591.98,32.9176 591.918,26.8985 591.855,24.4348 591.792,25.6311 591.729,30.5113 591.665,39.0146 591.6,50.9934 591.535,66.2131 591.47,84.3548 
 591.404,105.019 591.337,127.735 591.271,151.966 591.203,177.125 591.135,202.585 591.067,227.699 590.998,251.812 590.929,274.28 590.86,294.491 590.79,311.881 
 590.719,325.954 590.648,336.295 590.577,342.589 590.505,344.632 590.432,342.34 590.359,335.758 590.286,325.058 590.212,310.544 590.138,292.64 590.063,271.883 
 589.988,248.91 589.912,224.437 589.836,199.241 589.759,174.132 589.682,149.929 589.605,127.429 589.527,107.384 589.448,90.4686 589.37,77.2563 589.29,68.1963 
 589.21,63.5945 589.13,63.5991 589.049,68.1919 588.968,77.1854 588.886,90.227 588.804,106.809 588.722,126.287 588.639,147.901 588.555,170.804 588.471,194.097 
 588.387,216.864 588.302,238.205 588.216,257.282 588.131,273.347 588.044,285.78 587.958,294.116 587.87,298.066 587.783,297.528 587.695,292.602 587.606,283.576 
 587.517,270.923 587.427,255.274 587.337,237.391 587.247,218.132 587.156,198.412 587.065,179.154 586.973,161.248 586.881,145.504 586.788,132.614 586.695,123.119 
 586.601,117.376 586.507,115.546 586.413,117.586 586.318,123.253 586.222,132.116 586.126,143.589 586.03,156.958 585.933,171.428 585.836,186.168 585.738,200.36 
 585.64,213.245 585.541,224.172 585.442,232.629 585.342,238.276 585.242,240.959 585.142,240.716 585.041,237.768 584.94,232.501 584.838,225.432 584.735,217.169 
 584.633,208.369 584.529,199.681 584.426,191.705 584.322,184.947 584.217,179.778 584.112,176.416 584.007,174.91 583.901,175.142 583.794,176.842 583.687,179.621 
 583.58,183.006 583.472

# Bessel equation $$x^2 u'' + x u + (x^2 - \nu^2)u$$

In [4]:
x=Fun(identity,1..2000) 
d=domain(x)
B=dirichlet()  
@manipulate for ν=0:.1:2000
	L=x^2*𝒟^2 + x*𝒟 + (x^2 - ν^2)   # our differential operator
   	u=[B;L]\[besselj(ν,first(d));besselj(ν,last(d));0]
    plot(u;ylims=(-.2,.2))
end

Interact.Slider{Float64}(Signal{Float64}(1000.0, nactions=0),"ν",1000.0,0.0:0.1:2000.0,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 596.063,181.715 596.063,181.717 596.063,181.722 596.063,181.73 596.062,181.74 596.062,181.753 596.062,181.769 596.061,181.787 596.061,181.808 596.06,181.831 
 596.06,181.857 596.059,181.886 596.058,181.918 596.057,181.952 596.056,181.989 596.055,182.028 596.054,182.071 596.053,182.116 596.052,182.164 596.051,182.214 
 596.05,182.268 596.048,182.325 596.047,182.384 596.045,182.446 596.044,182.512 596.042,182.58 596.041,182.651 596.039,182.726 596.037,182.803 596.035,182.884 
 596.033,182.968 596.032,183.055 596.029,183.146 596.027,183.239 596.025,183.336 596.023,183.437 596.021,183.541 596.018,183.648 596.016,183.759 596.013,183.873 
 596.011,183.991 596.008,184.112 596.006,184.237 596.003,184.366 596,184.499 595.997,184.635 595.994,184.776 595.991,184.92 595.988,185.068 595.985,185.22 
 595.982,185.376 595.979,185.536 595.976,185.7 595.972,185.868 595.969,186.041 595.965,186.217 595.962,186.398 595.958,186.583 595.954,186.772 595.951,186.965 
 595.947,187.163 595.943,187.365 595.939,187.572 595.935,187.783 595.931,187.998 595.927,188.218 595.923,188.442 595.918,188.671 595.914,188.904 595.91,189.141 
 595.905,189.383 595.901,189.63 595.896,189.881 595.892,190.136 595.887,190.396 595.882,190.66 595.877,190.928 595.872,191.201 595.867,191.478 595.862,191.759 
 595.857,192.045 595.852,192.335 595.847,192.629 595.842,192.927 595.836,193.228 595.831,193.534 595.826,193.844 595.82,194.158 595.814,194.475 595.809,194.796 
 595.803,195.12 595.797,195.448 595.791,195.779 595.786,196.113 595.78,196.45 595.774,196.79 595.768,197.133 595.761,197.478 595.755,197.826 595.749,198.176 
 595.743,198.528 595.736,198.882 595.73,199.238 595.723,199.595 595.716,199.954 595.71,200.314 595.703,200.674 595.696,201.036 595.689,201.397 595.683,201.759 
 595.676,202.121 595.669,202.482 595.661,202.843 595.654,203.202 595.647,203.561 595.64,203.918 595.632,204.273 595.625,204.626 595.617,204.976 595.61,205.324 
 595.602,205.668 595.595,206.009 595.587,206.347 595.579,206.68 595.571,207.008 595.563,207.331 595.555,207.649 595.547,207.962 595.539,208.268 595.531,208.568 
 595.523,208.861 595.514,209.146 595.506,209.424 595.498,209.693 595.489,209.954 595.48,210.206 595.472,210.449 595.463,210.681 595.454,210.904 595.446,211.115 
 595.437,211.316 595.428,211.505 595.419,211.682 595.41,211.846 595.401,211.998 595.391,212.137 595.382,212.261 595.373,212.372 595.363,212.469 595.354,212.55 
 595.344,212.616 595.335,212.667 595.325,212.701 595.316,212.72 595.306,212.721 595.296,212.706 595.286,212.673 595.276,212.623 595.266,212.555 595.256,212.468 
 595.246,212.363 595.236,212.24 595.225,212.098 595.215,211.936 595.205,211.756 595.194,211.556 595.184,211.336 595.173,211.097 595.162,210.838 595.152,210.56 
 595.141,210.262 595.13,209.944 595.119,209.607 595.108,209.25 595.097,208.874 595.086,208.478 595.075,208.064 595.064,207.63 595.052,207.178 595.041,206.708 
 595.03,206.219 595.018,205.713 595.007,205.189 594.995,204.649 594.983,204.093 594.972,203.52 594.96,202.932 594.948,202.33 594.936,201.714 594.924,201.084 
 594.912,200.442 594.9,199.788 594.888,199.123 594.876,198.447 594.863,197.763 594.851,197.069 594.838,196.369 594.826,195.662 594.813,194.949 594.801,194.232 
 594.788,193.512 594.775,192.79 594.763,192.067 594.75,191.343 594.737,190.622 594.724,189.903 594.711,189.188 594.698,188.478 594.684,187.775 594.671,187.079 
 594.658,186.393 594.645,185.718 594.631,185.054 594.618,184.404 594.604,183.769 594.59,183.149 594.577,182.547 594.563,181.964 594.549,181.402 594.535,180.86 
 594.521,180.342 594.507,179.848 594.493,179.38 594.479,178.939 594.465,178.525 594.451,178.141 594.436,177.788 594.422,177.466 594.408,177.176 594.393,176.921 
 594.378,176

# Exponentially ill-conditioned Lee & Greengard BVP $\epsilon u'' - x u' + u = 0$

In [4]:
x=Fun(identity)
B=dirichlet()

@manipulate for ϵ=0.0005:0.0005:0.4
    L=ϵ*𝒟^2 - x*𝒟 + I

     u=[B;L]\[1.,2.]
    plot(u;ylims=(-2.,3.))
end

Interact.Slider{Float64}(Signal{Float64}(0.2, nactions=0),"ϵ",0.2,0.0005:0.0005:0.4,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 25.6518,154.793 25.769,154.989 26.0032,155.379 26.3545,155.962 26.8227,156.735 27.4075,157.693 28.1088,158.83 28.9262,160.141 29.8595,161.62 30.9081,163.257 
 32.0718,165.044 33.35,166.973 34.7422,169.034 36.2478,171.217 37.8662,173.511 39.5968,175.906 41.4388,178.392 43.3915,180.957 45.454,183.59 47.6256,186.282 
 49.9053,189.021 52.2922,191.798 54.7854,194.602 57.3837,197.423 60.0861,200.254 62.8916,203.083 65.7989,205.905 68.8069,208.709 71.9143,211.49 75.1199,214.24 
 78.4223,216.953 81.8202,219.623 85.3121,222.245 88.8968,224.814 92.5726,227.324 96.3381,229.774 100.192,232.158 104.132,234.474 108.157,236.719 112.265,238.891 
 116.455,240.987 120.725,243.006 125.073,244.947 129.497,246.808 133.996,248.589 138.567,250.289 143.209,251.907 147.92,253.443 152.698,254.897 157.541,256.269 
 162.447,257.558 167.414,258.766 172.439,259.893 177.522,260.938 182.659,261.903 187.85,262.787 193.09,263.592 198.379,264.318 203.714,264.965 209.093,265.535 
 214.514,266.027 219.975,266.443 225.473,266.783 231.006,267.047 236.572,267.237 242.168,267.353 247.793,267.396 253.443,267.366 259.117,267.263 264.812,267.089 
 270.526,266.844 276.257,266.528 282.002,266.142 287.759,265.686 293.525,265.161 299.299,264.568 305.077,263.905 310.857,263.175 316.638,262.377 322.416,261.511 
 328.19,260.578 333.956,259.579 339.713,258.512 345.458,257.379 351.188,256.18 356.903,254.914 362.598,253.583 368.272,252.185 373.922,250.721 379.547,249.191 
 385.143,247.595 390.709,245.934 396.242,244.206 401.74,242.413 407.2,240.553 412.621,238.627 418,236.635 423.336,234.577 428.625,232.453 433.865,230.263 
 439.055,228.006 444.193,225.683 449.275,223.294 454.301,220.838 459.268,218.317 464.174,215.73 469.017,213.078 473.795,210.361 478.506,207.579 483.148,204.733 
 487.719,201.825 492.218,198.855 496.642,195.824 500.99,192.733 505.26,189.585 509.45,186.38 513.558,183.122 517.583,179.813 521.523,176.455 525.377,173.052 
 529.142,169.607 532.818,166.125 536.403,162.608 539.895,159.063 543.293,155.495 546.595,151.909 549.8,148.311 552.908,144.708 555.916,141.108 558.823,137.518 
 561.629,133.947 564.331,130.402 566.929,126.893 569.423,123.43 571.809,120.023 574.089,116.681 576.261,113.415 578.323,110.236 580.276,107.154 582.118,104.182 
 583.849,101.329 585.467,98.6067 586.973,96.0259 588.365,93.5969 589.643,91.3299 590.807,89.2347 591.855,87.3204 592.789,85.5956 593.606,84.0682 594.307,82.7452 
 594.892,81.6328 595.36,80.7365 595.712,80.0604 595.946,79.6078 596.063,79.381 
 "/>
 
 
 
 
 y1

# Piecewise $$u'' + \omega u \begin{cases} 1 & |x| < c\cr 0 & {\rm otherwise}\end{cases}  = 0,\qquad u(-1)=1,u(1)=0$$

In [5]:
@manipulate for ω=1:50000,c=0.1:0.01:1.
    f=Fun(x->abs(x)<c?1:0,(-1..1) \ [-c,c])
    S=space(f)
    B=dirichlet(S)
    plot([B;𝒟^2+ω*f]\[1.,0.];ylims=(-0.5,1.1))
end

Interact.Slider{Int64}(Signal{Int64}(25000, nactions=0),"ω",25000,1:50000,"d",true)

Interact.Slider{Float64}(Signal{Float64}(0.55, nactions=0),"c",0.55,0.1:0.01:1.0,".3f",true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 43.4296,27.505 43.4323,27.5103 43.4378,27.5207 43.446,27.5364 43.457,27.5574 43.4707,27.5835 43.4872,27.6149 43.5063,27.6515 43.5283,27.6934 43.5529,27.7405 
 43.5804,27.7928 43.6105,27.8503 43.6434,27.913 43.679,27.981 43.7173,28.0541 43.7584,28.1325 43.8022,28.2161 43.8487,28.3049 43.898,28.3988 43.9499,28.498 
 44.0046,28.6024 44.062,28.7119 44.1222,28.8267 44.185,28.9466 44.2505,29.0716 44.3188,29.2019 44.3898,29.3373 44.4634,29.4778 44.5398,29.6235 44.6188,29.7743 
 44.7005,29.9303 44.785,30.0914 44.8721,30.2576 44.9618,30.4289 45.0543,30.6054 45.1494,30.7869 45.2472,30.9735 45.3477,31.1652 45.4508,31.362 45.5566,31.5638 
 45.665,31.7707 45.7761,31.9826 45.8898,32.1996 46.0061,32.4216 46.1251,32.6486 46.2467,32.8806 46.3709,33.1177 46.4977,33.3597 46.6271,33.6066 46.7591,33.8586 
 46.8938,34.1155 47.031,34.3773 47.1708,34.6441 47.3132,34.9157 47.4581,35.1923 47.6056,35.4738 47.7557,35.7602 47.9083,36.0514 48.0635,36.3475 48.2212,36.6485 
 48.3814,36.9542 48.5442,37.2648 48.7095,37.5802 48.8772,37.9004 49.0475,38.2253 49.2203,38.555 49.3956,38.8895 49.5733,39.2287 49.7536,39.5726 49.9362,39.9212 
 50.1214,40.2745 50.309,40.6324 50.499,40.995 50.6915,41.3623 50.8863,41.7342 51.0836,42.1107 51.2833,42.4917 51.4854,42.8774 51.6899,43.2676 51.8968,43.6623 
 52.106,44.0616 52.3176,44.4653 52.5315,44.8736 52.7478,45.2863 52.9664,45.7034 53.1874,46.125 53.4106,46.551 53.6362,46.9814 53.864,47.4162 54.0941,47.8553 
 54.3265,48.2988 54.5612,48.7466 54.7981,49.1987 55.0373,49.655 55.2786,50.1156 55.5222,50.5805 55.768,51.0495 56.0161,51.5228 56.2663,52.0002 56.5186,52.4818 
 56.7732,52.9675 57.0299,53.4573 57.2887,53.9513 57.5497,54.4492 57.8127,54.9513 58.0779,55.4573 58.3452,55.9674 58.6146,56.4814 58.8861,56.9994 59.1596,57.5213 
 59.4351,58.0471 59.7127,58.5768 59.9923,59.1104 60.274,59.6478 60.5576,60.189 60.8432,60.734 61.1308,61.2828 61.4203,61.8354 61.7118,62.3916 62.0053,62.9515 
 62.3006,63.5152 62.5979,64.0824 62.8971,64.6533 63.1981,65.2278 63.5011,65.8058 63.8058,66.3874 64.1125,66.9725 64.4209,67.5611 64.7312,68.1532 65.0433,68.7487 
 65.3571,69.3476 65.6728,69.9499 65.9902,70.5555 66.3093,71.1645 66.6302,71.7769 66.9528,72.3924 67.2771,73.0113 67.6031,73.6334 67.9307,74.2586 68.2601,74.8871 
 68.5911,75.5186 68.9237,76.1533 69.2579,76.7911 69.5937,77.432 69.9312,78.0758 70.2702,78.7227 70.6107,79.3726 70.9528,80.0254 71.2964,80.6811 71.6416,81.3397 
 71.9882,82.0011 72.3363,82.6654 72.6859,83.3325 73.0369,84.0024 73.3894,84.675 73.7433,85.3503 74.0986,86.0283 74.4553,86.7089 74.8134,87.3922 75.1728,88.078 
 75.5336,88.7665 75.8957,89.4574 76.2591,90.1509 76.6238,90.8468 76.9897,91.5451 77.357,92.2459 77.7255,92.9491 78.0952,93.6546 78.4661,94.3624 78.8382,95.0725 
 79.2115,95.7848 79.586,96.4994 79.9616,97.2161 80.3384,97.935 80.7162,98.6561 81.0952,99.3792 81.4752,100.104 81.8563,100.832 82.2385,101.561 82.6217,102.292 
 83.0059,103.025 83.391,103.76 83.7772,104.497 84.1644,105.236 84.5524,105.976 84.9414,106.719 85.3314,107.463 85.7222,108.209 86.1139,108.956 86.5064,109.705 
 86.8998,110.456 87.294,111.208 87.6891,111.962 88.0849,112.717 88.4815,113.474 88.8788,114.232 89.2769,114.992 89.6757,115.753 90.0752,116.515 90.4754,117.279 
 90.8762,118.044 91.2778,118.81 91.6799,119.577 92.0826,120.346 92.486,121.115 92.8899,121.886 93.2944,122.658 93.6994,123.431 94.105,124.205 94.5111,124.98 
 94.9176,125.755 95.3246,126.532 95.7321,127.31 96.14,128.088 96.5483,128.867 96.9571,129.647 97.3662,130.428 97.7757,131.209 98.1855,131.991 98.5956,132.774 
 99.0061,133.557 99.4168,134.341 99.8278,135.125 100.239,135.91 100.651,136.695 101.062,137.481 101.474,138.267 101.886,139.053 102.299,139.84 102.711,140.627 
 103.124,141.414 103.536,1

# Forced Helmholtz equation $$u_{xx} + u_{yy} + k u = {\rm e}^{-(x-x_0)^2-(y-y_0)^2},\qquad \partial u = 0$$

In [6]:
plotly()

d=Interval()^2
B=Dirichlet(d)
Δ=Laplacian(d)

k = slider(-100.:.1:100.; label="k")
x₀ = slider(-2.:0.1:2.; label="x₀")
y₀ = slider(-2.:0.1:2.; label="y₀")

QR = map(k->qrfact([B;Δ+k*I]),signal(k))
f = map((x0,y0)->Fun((x,y)->exp(-(x-x0)^2-(y-y0)^2)), signal(x₀), signal(y₀)) 
u = map((QR,f)->\(QR,[0.0;f];tolerance=1E-4),QR,f)

map(display, [k,x₀,y₀]) #display widgets
display(map(contour,u)) #display results;

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"k",0.0,-100.0:0.1:100.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"x₀",0.0,-2.0:0.1:2.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"y₀",0.0,-2.0:0.1:2.0,".3f",true)

In [12]:
plotly()

d=Interval()^2
B=[dirichlet(d[1])⊗I;I⊗ldirichlet(d[2]);I⊗rneumann(d[2])]
Δ=Laplacian(d)

k = slider(-100.:.1:100.; label="k")
x₀ = slider(-2.:0.1:2.; label="x₀")
y₀ = slider(-2.:0.1:2.; label="y₀")

QR = map(k->qrfact([B;Δ+k*I]),signal(k))
f = map((x0,y0)->Fun((x,y)->exp(-(x-x0)^2-(y-y0)^2),d), signal(x₀), signal(y₀)) 
u = map((QR,f)->\(QR,[0.0;0.0;0.0;0.0;f];tolerance=1E-4),QR,f)

map(display, [k,x₀,y₀]) #display widgets
display(map(contour,u)) #display results;

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"k",0.0,-100.0:0.1:100.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"x₀",0.0,-2.0:0.1:2.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=1),"y₀",0.0,-2.0:0.1:2.0,".3f",true)

#     Dirichlet Helmholtz equation $$\Delta u + k u = 0, u(\pm 1,y)=u(x,\pm 1) = 1$$

In [13]:
d=Interval()^2
B=Dirichlet(d)
Δ=Laplacian(d)

@manipulate for k=-200.0:.001:200.0
    contour(\([B;Δ+k*I],[ones(∂(d));0.];tolerance=1E-3))
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"k",0.0,-200.0:0.001:200.0,".3f",true)

# Convection diffusion $u_t = \epsilon u_{xx} + (B+C x) u_x, u(x,0)=e^{-20x^2}, u(-1,t)=u(1,t) = 0$

In [14]:
dx=Interval();dt=Interval(0,1.)
d=dx*dt
Dx=Derivative(d,[1,0]);Dt=Derivative(d,[0,1])
x,t=Fun(dx*dt)
Bcs=timedirichlet(d)
u0=Fun(x->exp(-20x^2),dx)
@manipulate for ε=0.001:0.001:2.,B=-5.:0.1:5.,C=-5.:0.1:5.
    V=B+C*x    
    u=\([Bcs;Dt-ε*Dx^2-V*Dx],[u0;zeros(3)];tolerance=1E-3,maxlength=2000)
    contour(u;xlims=(-1.,1.),ylims=(0.,1.))
end

Interact.Slider{Float64}(Signal{Float64}(1.0, nactions=0),"ε",1.0,0.001:0.001:2.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"B",0.0,-5.0:0.1:5.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"C",0.0,-5.0:0.1:5.0,".3f",true)

# Schrodinger equation $$i \epsilon u_t = -{\epsilon^2 \over 2} u_{xx} + x^2 u$$

In [15]:
dx=Interval(0.,1.);dt=Interval(0.0,0.54)
d=dx*dt
x,t=Fun(d)
V=x^2

Dt=Derivative(d,[0,1]);Dx=Derivative(d,[1,0])

@manipulate for ϵ=0.005:0.002:0.3
    u0=Fun(x->exp(-25*(x-0.5)^2)*exp(-1.0im/(5*ϵ)*log(2cosh(5*(x-0.5)))),dx)
    L=1im*ϵ*Dt+0.5*ϵ^2*Dx^2-V

    u=\([timedirichlet(d);L],[u0;zeros(3)];tolerance=1E-3,maxlength=2000)
    contour(real(u))
end

Interact.Slider{Float64}(Signal{Float64}(0.151, nactions=0),"ϵ",0.151,0.005:0.002:0.299,".3f",true)

# Nonlinear BVP $$\epsilon u'' + 6(1-x^2)u' +u^2=1$$ $$u(-1)=u(1)=0$$

In [17]:
x=Fun()
u0=0.0x
@manipulate for ε=0.001:0.001:1.,c=0.:0.1:1.
    N=u->[u(-1.)-c;u(1.);ε*u''+6*(1-x^2)*u'+u^2-1.]

    u=newton(N,u0)
    plot(u;ylims=(-1.,1.))
end

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=0),"ε",0.5,0.001:0.001:1.0,".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.5, nactions=0),"c",0.5,0.0:0.1:1.0,".3f",true)